In [10]:
#Celda para librerías
import sklearn as sk


import matplotlib as plt
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.model_selection import StratifiedKFold

# LightGBM
from lightgbm import LGBMClassifier
import lightgbm as lgb



from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score


import json

import warnings
warnings.filterwarnings('ignore')

print(np.__version__)
from ydata_profiling import ProfileReport
import os
print("Directorio actual:", os.getcwd())

# Have all columns appear when dataframes are displayed.
pd.set_option('display.max_columns', None) 
# Have 100 rows appear when a dataframe is displayed
pd.set_option('display.max_rows', 500)
# Display dimensions whenever a dataframe is printed out.
pd.set_option('display.show_dimensions', True)

#Importando los datos

#app_train_def = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/application_train_preprocesado_definitivo_v1.csv')
#app_train_def_2 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v2.csv')
#app_train_def_3 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v3.csv')
#app_train_def_4 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v4.csv')
#app_train_def_5 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v5.csv')
#app_train_def_6 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v6.csv')
app_train_def_7 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v7.csv')
#app_train_def_7_v2 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/app_def_v7_menor_dimension.csv')
#print(app_train_def_5.shape)
#print(app_train_def_6.shape)
print(app_train_def_7.shape)
#print(app_train_def_7_v2.shape)

1.26.4
Directorio actual: /home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS
(307511, 1323)


In [12]:
#Train y Test
X = app_train_def_7.drop(columns = ['TARGET'])
y = app_train_def_7['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train.columns = X_train.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
X_test.columns = X_test.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

In [ ]:
#Definimos espacio de hiperparámetros para LightGBM
space_lightGBM = {
    'boosting_type': 'gbdt',  # Fijo
    'objective': 'binary',   # Fijo
    'metric': 'auc',         # Fijo
    ##AJUSTAR MAX DEPTH
    'num_leaves': hp.quniform('num_leaves', 25, 45, 1), 
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.03),  
    'min_child_samples': hp.quniform('min_child_samples', 70, 100, 1),  
    'min_child_weight': hp.uniform('min_child_weight', 0.001, 0.02),
    'min_gain_to_split': hp.uniform('min_gain_to_split', 0.001, 0.1),
    'max_bin': hp.quniform('max_bin', 220, 350, 10),
    'feature_fraction': hp.uniform('feature_fraction', 0.15, 0.5),
    'max_depth': hp.choice('max_depth', [-1, 3, 5]), 
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1.0), #No admite valores mayores a 1.0
    'bagging_freq': hp.quniform('bagging_freq', 7, 15, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 70),  
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 0.2),  
    
    'scale_pos_weight': 1,  # Fijo
    'is_unbalance': False  # Fijo
}

Trials_lightGBM_Ligero = Trials()


In [ ]:
n_iter = 400

def F_OPT(params):
    params['max_bin'] = int(params['max_bin'])
    params['num_leaves'] = int(params['num_leaves'])
    params['min_child_samples'] = int(params['min_child_samples'])
    params['bagging_freq'] = int(params['bagging_freq'])
    params['min_child_samples'] = int(params['min_child_samples'])

    model = LGBMClassifier(
        n_estimators=n_iter,
        boosting_type=params['boosting_type'],
        objective=params['objective'],
        metric=params['metric'],
        num_leaves=params['num_leaves'],
        learning_rate=params['learning_rate'],
        min_child_samples=params['min_child_samples'],
        min_child_weight=params['min_child_weight'],
        min_gain_to_split=params['min_gain_to_split'],
        max_bin=params['max_bin'],
        feature_fraction=params['feature_fraction'],
        max_depth=params['max_depth'],
        bagging_fraction=params['bagging_fraction'],
        bagging_freq=params['bagging_freq'],
        reg_lambda=params['reg_lambda'],
        reg_alpha=params['reg_alpha'],
        scale_pos_weight=params['scale_pos_weight'],
        is_unbalance=params['is_unbalance'],
        random_state=42)
    
    cv_Strat = StratifiedKFold(n_splits=5)
    auc_lightGBM_Ligero = cross_val_score(model, X_train, y_train, cv=cv_Strat, scoring='roc_auc').mean()
    return {'loss': -auc_lightGBM_Ligero, 'status': STATUS_OK}
    

In [ ]:
%%capture
best_lightGBM_Ligero = fmin(fn=F_OPT,
                            space=space_lightGBM,
                            algo=tpe.suggest,
                            max_evals=50,
                            trials=Trials_lightGBM_Ligero,
                            rstate= np.random.default_rng(42)
)

best_lightGBM_Ligero['num_leaves'] = int(best_lightGBM_Ligero['num_leaves'])
best_lightGBM_Ligero['max_bin'] = int(best_lightGBM_Ligero['max_bin'])
best_lightGBM_Ligero['min_child_samples'] = int(best_lightGBM_Ligero['min_child_samples'])
best_lightGBM_Ligero['bagging_freq'] = int(best_lightGBM_Ligero['bagging_freq'])
best_lightGBM_Ligero['min_child_samples'] = int(best_lightGBM_Ligero['min_child_samples'])

#print(best_lightGBM_Ligero)

In [ ]:
import pickle
#with open("Trials_LightGBM_Ligero_v7_server_50_evals_rango_ampliado.pkl", "wb") as f:
#    pickle.dump(Trials_lightGBM_Ligero, f)
with open("/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/Trials/Trials_LightGBM_Ligero_v7_server_50_evals_rango_ampliado.pkl", "rb") as f:
    Trials_lightGBM_Ligero = pickle.load(f)

In [ ]:
#Mostramos los resultados de los trials 
from tabulate import tabulate

def trials_to_dataframe(trials):
    """
    Convierte la info de 'trials' de Hyperopt en un DataFrame 
    con columnas relevantes (loss, hiperparámetros, etc.).
    """
    rows = []
    for trial in trials.trials:
        # trial['result']['loss'] -> la métrica
        # trial['misc']['vals'] -> diccionario de hiperparámetros propuestos
        loss = trial['result']['loss']
        vals = trial['misc']['vals']
        
        # Convertir vals a algo "plano"
        row = {**vals}
        row['loss'] = loss
        rows.append(row)
    
    # df con columnas = keys (vals + 'loss')
    df = pd.DataFrame(rows)
    
    # A veces los hps que eran hp.quniform se quedan en listas => df['num_leaves'].apply(lambda x: x[0] if isinstance(x, list) else x)
    for c in df.columns:
        df[c] = df[c].apply(lambda x: x[0] if isinstance(x, list) else x)
    
    return df

# Para usarlo:
results_df = trials_to_dataframe(Trials_lightGBM_Ligero)
# Ordenamos por 'loss' asc => AUC mayor es 'loss' menor
results_df_sorted = results_df.sort_values(by='loss', ascending=True)
best_10 = results_df_sorted.head(10)
print(tabulate(best_10, headers='keys', tablefmt='psql'))

In [ ]:
#Vamos a realizar una selección de variables previa a una optimización más completa y una búsqueda SS
# más exhaustiva de hiperparámetros. Para ello, vamos a utilizar Permutation Importance.
from sklearn.inspection import permutation_importance

# Extraer el mejor trial en forma de diccionario
best_trial = results_df_sorted.iloc[0].to_dict()

for key in ['num_leaves', 'max_bin', 'min_child_samples', 'bagging_freq', 'max_depth']:
    best_trial[key] = int(best_trial[key])
# Eliminar la clave 'loss' ya que no es un hiperparámetro para el modelo
best_trial.pop('loss', None)

# Si es necesario, puedes asegurarte de que los valores sean del tipo correcto,
# aunque en nuestro proceso ya se han aplanado.

# Crear el modelo utilizando el operador ** para desempaquetar el diccionario de hiperparámetros
model_ligero = LGBMClassifier(
    n_estimators=n_iter,          # Puedes dejar este parámetro fijo o incluirlo en best_trial
    boosting_type='gbdt',         # Fijo
    objective='binary',           # Fijo
    metric='auc',                 # Fijo
    scale_pos_weight=1,           # Fijo
    is_unbalance=False,           # Fijo
    random_state=42,              # Fijo
    **best_trial                 # Desempaqueta los hiperparámetros encontrados
)

# Ajustar el modelo
model_ligero.fit(X_train, y_train)



In [ ]:
%%capture
pi = permutation_importance(
    estimator=model_ligero,
    X=X_train,
    y=y_train,
    scoring='roc_auc',
    n_repeats=20,
    random_state=42
    #Probar n jobs
)

In [ ]:
import pickle
#with open("pi_lightGBM_Ligero_v7_server_20_reapeats.pkl", "wb") as f:
#    pickle.dump(pi, f)
#TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/Permutation_Pickle/pi_lightGBM_Ligero_v7_server_20_reapeats.pkl
with open("/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/Permutation_Pickle/pi_lightGBM_Ligero_v7_server_20_reapeats.pkl", "rb") as f:
    pi = pickle.load(f)


In [ ]:
from tabulate import tabulate

features = X_train.columns
importances_mean = pi.importances_mean
importances_std = pi.importances_std

#Lo unimos en un DF

df_pi = pd.DataFrame({
    'feature': features,
    'importance_mean': importances_mean,
    'importance_std': importances_std
})
df_pi.to_csv("Permutation_Importance_LightGBM_Ligero_v7_server_20_repeats.csv", index=False)

df_pi_sorted = df_pi.sort_values(by='importance_mean', ascending=False)
print(tabulate(df_pi_sorted, headers='keys', tablefmt='psql'))

In [ ]:
subset_features_pi = df_pi_sorted[df_pi_sorted['importance_mean'] > 0]
subset_features= subset_features_pi['feature'].tolist()
X_train_subset = X_train[subset_features]
y_train_subset = y_train
X_test_subset = X_test[subset_features]
y_test_subset = y_test
#print(X_train_subset.describe())

In [30]:
"""
space_lightGBM_completo = {
    'boosting_type': 'gbdt',  # Fijo
    'objective': 'binary',   # Fijo
    'metric': 'auc',         # Fijo
    #Ajustamos los HP para centrarlos y reducir el rango a los resultados obtenidos anteriormente
    'num_leaves': hp.quniform('num_leaves', 35, 45, 1), 
    'learning_rate': hp.uniform('learning_rate', 0.015, 0.03),  
    'min_child_samples': hp.quniform('min_child_samples', 80, 110, 1),  
    'min_child_weight': hp.uniform('min_child_weight', 0.01, 0.015),
    'min_gain_to_split': hp.uniform('min_gain_to_split', 0.02, 0.1),
    'max_bin': hp.quniform('max_bin', 250, 300, 5),
    'feature_fraction': hp.uniform('feature_fraction', 0.1, 0.4),
    'max_depth': hp.choice('max_depth', [-1, 3]), 
    'bagging_fraction': hp.uniform('bagging_fraction', 0.6, 1.0),
    'bagging_freq': hp.quniform('bagging_freq', 10, 18, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 50),  
    'reg_alpha': hp.uniform('reg_alpha', 0.05, 0.15),  
    
    'scale_pos_weight': 1,  # Fijo
    'is_unbalance': False  # Fijo
}
"""
## PRUEBAS PARA CONTROLAR LOS PROBLEMAS CON LA PROFUNDIDAD
# Tras varias pruebas e investigaciones, voy a probar primero ciertas cosas que me interesan en el csv reducido y luego al completo
space_lightGBM_completo = {
    'boosting_type': 'gbdt',          # Fijo
    'objective': 'binary',            # Fijo
    'metric': 'auc',                  # Fijo
    # Ajustamos el rango de num_leaves para permitir mayor flexibilidad, aunque puede que nos provoque overfitting, así que vamos a probar y vamos viendo
    'num_leaves': hp.quniform('num_leaves', 35, 45, 1),
    'learning_rate': hp.uniform('learning_rate', 0.015, 0.018),
    'min_child_samples': hp.quniform('min_child_samples', 100, 110, 1),
    'min_child_weight': hp.uniform('min_child_weight', 0.01, 0.015),
    'min_gain_to_split': hp.uniform('min_gain_to_split', 0.02, 0.03),
    'max_bin': hp.quniform('max_bin', 290, 310, 5),
    'feature_fraction': hp.uniform('feature_fraction', 0.1, 0.15),
    'n_estimators': hp.uniform('n_estimators', 1900,2200,100),
    # Permitimos explorar tanto árboles sin límite como con profundidad controlada
    'max_depth': hp.choice('max_depth', [-1]),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.85, 0.9),
    'bagging_freq': hp.quniform('bagging_freq', 15, 18, 1),
    'reg_lambda': hp.uniform('reg_lambda', 30, 40),
    'reg_alpha': hp.uniform('reg_alpha', 0.09, 0.11),
    'n_estimators': hp.quniform('n_estimators', 2000, 2300, 100),
    'scale_pos_weight': 1,            # Fijo
    'is_unbalance': False             # Fijo
}
Trials_lightGBM_Completo = Trials()

In [31]:
n_iter = 2000
#Vamos a probar ahora a realizar esa búsqueda más extensa pero con seleccionando las variables que parecen tener mayor releveancia
def F_OPT_Completa(params):
    params['max_bin'] = int(params['max_bin'])
    params['min_child_samples'] = int(params['min_child_samples'])
    params['bagging_freq'] = int(params['bagging_freq'])
    params['min_child_samples'] = int(params['min_child_samples'])
    params['num_leaves'] = int(params['num_leaves'])
    params['n_estimators'] = int(params['n_estimators'])

    depth = params['max_depth']
    num_leaves = params['num_leaves']

    # Si max_depth es positivo (por ej. 4, 5, 6...), forzamos la regla:
    # num_leaves <= 2^depth
    """
    if depth > 0:
        max_leaves_allowed = 2 ** depth
        if num_leaves > max_leaves_allowed:
            num_leaves = max_leaves_allowed

    # Asignamos de vuelta
    params['max_depth'] = depth
    params['num_leaves'] = num_leaves
    """
    model = LGBMClassifier(
        n_estimators=params['n_estimators'],
        #early_stopping_round=50,
        boosting_type=params['boosting_type'],
        objective=params['objective'],
        metric=params['metric'],
        num_leaves=params['num_leaves'],
        learning_rate=params['learning_rate'],
        min_child_samples=params['min_child_samples'],
        min_child_weight=params['min_child_weight'],
        min_gain_to_split=params['min_gain_to_split'],
        max_bin=params['max_bin'],
        feature_fraction=params['feature_fraction'],
        max_depth=params['max_depth'],
        bagging_fraction=params['bagging_fraction'],
        bagging_freq=params['bagging_freq'],
        reg_lambda=params['reg_lambda'],
        reg_alpha=params['reg_alpha'],
        scale_pos_weight=params['scale_pos_weight'],
        is_unbalance=params['is_unbalance'],
        random_state=42,
        n_jobs=20
        )
    
    cv_Strat = StratifiedKFold(n_splits=5)
    auc_lightGBM_Completo = cross_val_score(model, X_train, y_train, cv=cv_Strat, scoring='roc_auc').mean()
    return {'loss': -auc_lightGBM_Completo, 'status': STATUS_OK}
    

In [32]:
%%capture
best_lightGBM_Completo = fmin(fn=F_OPT_Completa,
                            space=space_lightGBM_completo,
                            algo=tpe.suggest,
                            max_evals=50,
                            trials=Trials_lightGBM_Completo,
                            rstate= np.random.default_rng(42)
)
best_lightGBM_Completo['max_bin'] = int(best_lightGBM_Completo['max_bin'])
best_lightGBM_Completo['min_child_samples'] = int(best_lightGBM_Completo['min_child_samples'])
best_lightGBM_Completo['bagging_freq'] = int(best_lightGBM_Completo['bagging_freq'])
best_lightGBM_Completo['min_child_samples'] = int(best_lightGBM_Completo['min_child_samples'])
best_lightGBM_Completo['num_leaves'] = int(best_lightGBM_Completo['num_leaves'])
best_lightGBM_Completo['n_estimators'] = int(best_lightGBM_Completo['n_estimators'])


In [34]:
import pickle
with open("Trials_LightGBM_Completo_v7_ULTIMA_PRUEBA", "wb") as f:
    pickle.dump(Trials_lightGBM_Completo, f)

#with open("/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/Trials/Trials_LightGBM_Completo_v7_server_100_evals_csv_completo.pkl", "rb") as f:
#    Trials_lightGBM_Completo = pickle.load(f)

In [35]:
#Mostramos los resultados de los trials 
from tabulate import tabulate
import pandas as pd
def trials_to_dataframe(trials):
    rows = []
    for trial in trials.trials:
        if 'loss' not in trial['result']:
            # saltar este trial
            continue

        loss = trial['result']['loss']
        vals = trial['misc']['vals']
        
        row = {**vals}
        row['loss'] = loss
        rows.append(row)
    
    df = pd.DataFrame(rows)

    for c in df.columns:
        df[c] = df[c].apply(lambda x: x[0] if isinstance(x, list) else x)

    return df


# Para usarlo:
results_df = trials_to_dataframe(Trials_lightGBM_Completo)
# Ordenamos por 'loss' asc => AUC mayor es 'loss' menor
results_df_sorted = results_df.sort_values(by='loss', ascending=True)
best_10 = results_df_sorted.head(10)
print(tabulate(best_10, headers='keys', tablefmt='psql'))

+----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+----------------+--------------+-------------+--------------+-----------+
|    |   bagging_fraction |   bagging_freq |   feature_fraction |   learning_rate |   max_bin |   max_depth |   min_child_samples |   min_child_weight |   min_gain_to_split |   n_estimators |   num_leaves |   reg_alpha |   reg_lambda |      loss |
|----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+----------------+--------------+-------------+--------------+-----------|
| 32 |           0.874354 |             17 |           0.111451 |       0.0153463 |       300 |           0 |                 104 |          0.0121249 |           0.023562  |           2200 |           35 |   0.0982616 |      31.6216 | -0.794787 |
|  4 |  

In [37]:

# Extraer el mejor trial en forma de diccionario

best_trial = results_df_sorted.iloc[0].to_dict()

for key in ['num_leaves', 'max_bin', 'min_child_samples', 'bagging_freq', 'max_depth', 'n_estimators']:
    best_trial[key] = int(best_trial[key])
# Eliminar la clave 'loss' ya que no es un hiperparámetro para el modelo
best_trial.pop('loss', None)

print(best_trial)

# Crear el modelo utilizando el operador ** para desempaquetar el diccionario de hiperparámetros
final_model_lgb = LGBMClassifier(        
    boosting_type='gbdt',         # Fijo
    objective='binary',           # Fijo
    metric='auc',                 # Fijo
    scale_pos_weight=1,           # Fijo
    is_unbalance=False,           # Fijo
    random_state=42,              # Fijo
    **best_trial
)


final_model_lgb.fit(X_train, y_train)

"""
Optimización Completa:
|    |   bagging_fraction |   bagging_freq |   feature_fraction |   learning_rate |   max_bin |   max_depth |   min_child_samples |   min_child_weight |   min_gain_to_split |   num_leaves |   reg_alpha |   reg_lambda |      loss |
|----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------|
| 69 |           0.891186 |             16 |           0.106319 |       0.0173115 |       300 |           0 |                 101 |          0.0137934 |           0.0269529 |           41 |   0.0906638 |      31.1379 | -0.794675
"""

{'bagging_fraction': 0.8743538556507691, 'bagging_freq': 17, 'feature_fraction': 0.11145097371066257, 'learning_rate': 0.015346271469888557, 'max_bin': 300, 'max_depth': 0, 'min_child_samples': 104, 'min_child_weight': 0.012124933383198718, 'min_gain_to_split': 0.023561958354995503, 'n_estimators': 2200, 'num_leaves': 35, 'reg_alpha': 0.09826158385027922, 'reg_lambda': 31.621566021026435}
[LightGBM] [Warning] bagging_freq is set=17, subsample_freq=0 will be ignored. Current value: bagging_freq=17
[LightGBM] [Warning] min_gain_to_split is set=0.023561958354995503, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.023561958354995503
[LightGBM] [Warning] feature_fraction is set=0.11145097371066257, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.11145097371066257
[LightGBM] [Warning] bagging_fraction is set=0.8743538556507691, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8743538556507691
[LightGBM] [Warning] bagging_freq is 

'\nOptimización Completa:\n|    |   bagging_fraction |   bagging_freq |   feature_fraction |   learning_rate |   max_bin |   max_depth |   min_child_samples |   min_child_weight |   min_gain_to_split |   num_leaves |   reg_alpha |   reg_lambda |      loss |\n|----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------|\n| 69 |           0.891186 |             16 |           0.106319 |       0.0173115 |       300 |           0 |                 101 |          0.0137934 |           0.0269529 |           41 |   0.0906638 |      31.1379 | -0.794675\n'

In [38]:
y_pred_proba = final_model_lgb.predict_proba(X_test)[:, 1]
from sklearn.metrics import roc_auc_score
auc_test = roc_auc_score(y_test, y_pred_proba)
print(f'AUC en test: {auc_test}')

[LightGBM] [Warning] bagging_freq is set=17, subsample_freq=0 will be ignored. Current value: bagging_freq=17
[LightGBM] [Warning] min_gain_to_split is set=0.023561958354995503, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.023561958354995503
[LightGBM] [Warning] feature_fraction is set=0.11145097371066257, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.11145097371066257
[LightGBM] [Warning] bagging_fraction is set=0.8743538556507691, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8743538556507691
AUC en test: 0.7980380652469227


-AUC solo con app_train:
0.7687757918575168

-AUC añadiendo bureau.csv con threshhold de 0.00015:
0.7723138876162285

-AUC con threshold de 0.0001 y 40 features:
0.7741547798044517

-AUC con threshold de 0 y ~80 features:
0.777795871108371

Comienzo de pruebas en server
-Realizando Trials con 50 evaluaciones con todo el csv al completo el mejor resultado es de:

| 36 |           0.954169 |              4 |           0.316205 |       0.0289061 |       260 |           0 |                  62 |         0.00154331 |           0.0661147 |           39 |  0.134166   |      58.288  | -0.775321 |

Si probamos esta misma configuración sobre el test, es decir, HP con 50 evals y todos los features obtenemos: 
-AUC en test: 0.7807744950085171

-Realizando Trials con  100 evaluaciones con un rango más acotado de HP acorde a las pruebas anteriores, además de realizando una selección de variables con un threshold >0 (443 features) obtenemos:

|    |   bagging_fraction |   bagging_freq |   feature_fraction |   learning_rate |   max_bin |   max_depth |   min_child_samples |   min_child_weight |   min_gain_to_split |   num_leaves |   reg_alpha |   reg_lambda |      loss |
|----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------|
| 66 |           0.947598 |              5 |           0.256301 |       0.016433  |       270 |           0 |                  61 |         0.00531868 |           0.0774823 |           37 |   0.053589  |      37.9963 | -0.779018

Si probamos esta configuración sobre el test obtenemos:
-AUC en test: 0.7806464630531091 ---- Podemos observar que tenemos un resultado minimamente peor, pero puede deberse a fluctuaciones aleatorios, haber excluido alguena feature un poco útil... Aunque al ser una diferencia de ±0.0001–0.0002 de AUC se puede considerar que se encuentra dentro del margen de ruido.

-AUC en test: 0.7815232040819751 He puesto nstimators en 2000 en vez de 1000, que se me había pasado cambiarlo

Si añadimos los datos de la tabla bureau_balance y realizamos una optimización inicial (50), seleccionamos variables con threshold >0 (478 variables) y luego una optimizacón final (100), obtenemos un resultado de:
-AUC en test: 0.780960248215274

Si realizamos las pruebas sobre todo el csv sin realizar una selección de variables previa obtenemos:

| 65 |           0.946932 |             10 |           0.202777 |       0.0172655 |       250 |           0 |                  86 |         0.00728436 |           0.0451036 |           33 |   0.0558485 |      35.8496 | -0.779158 |

Y si realizamos las pruebas sobre el test:
-AUC en test: 0.7813628078996644

Tras añadir la tabla previous_application obtenemos con una primera optimización el mejor resultado obtenido es:

|    |   bagging_fraction |   bagging_freq |   feature_fraction |   learning_rate |   max_bin |   max_depth |   min_child_samples |   min_child_weight |   min_gain_to_split |   num_leaves |   reg_alpha |   reg_lambda |      loss |
|----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------|
| 39 |           0.758684 |             12 |           0.166488 |       0.0289361 |       250 |           0 |                  96 |          0.0115629 |          0.0289674  |           40 |   0.108431  |      31.6312 | -0.783675

Vamos a realizar diferentes pruebas con el número de variables para ver como varían los resulta

En primer lugar, realizando una optimización completa con un espacio reducido centrado en los mejores valores de la optimización ligera, hemos obtenido un resultado de:
 0.798114 |             14 |           0.206931 |       0.0162622 |       230 |           0 |                  94 |          0.0166739 |           0.084112  |           39 |   0.106233  |      37.0253 | -0.787348 |

Y si hacemos las pruebas sobre el conjunto de test, el resultado obtenido es:
-AUC en test: 0.790970334383

Tras una primera prueba con variables seleccionadas, threshold >0, obtenemos un resultado de:
0.799599 |             14 |           0.25449  |       0.0150386 |       230 |           0 |                  88 |          0.0153521 |           0.0475698 |           37 |   0.0846783 |      57.3954 | -0.787126 |
Y si realizamos las pruebas sobre el conjunto de test:
-AUC en test: 0.7901388712105
 Tras añadir el csv v5 hemos obtenido un resultado de la optimización completa con todo el csv de:
 |    |   bagging_fraction |   bagging_freq |   feature_fraction |   learning_rate |   max_bin |   max_depth |   min_child_samples |   min_child_weight |   min_gain_to_split |   num_leaves |   reg_alpha |   reg_lambda |      loss |
|----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------|
| 77 |           0.75008  |             14 |           0.155811 |       0.0171274 |       255 |           0 |                  93 |          0.0159414 |           0.073965  |           44 |   0.0658362 |      39.2701 | -0.788679

Y si realizamos estas pruebas sobre el test:
AUC en test: 0.792857925817487


Tras añadir el csv v6 y realizar una prueba ligera sobre el csv completo obtenemos:
|    |   bagging_fraction |   bagging_freq |   feature_fraction |   learning_rate |   max_bin |   max_depth |   min_child_samples |   min_child_weight |   min_gain_to_split |   num_leaves |   reg_alpha |   reg_lambda |      loss |
|----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------|
| 31 |           0.997003 |             15 |           0.177032 |       0.0298061 |       270 |           0 |                 100 |         0.0106796  |           0.0802473 |           43 |   0.115772  |      41.9076 | -0.789829

Con un espacio más centrado en esta anterior optimización y más evaluaciones obtenemos:
|    |   bagging_fraction |   bagging_freq |   feature_fraction |   learning_rate |   max_bin |   max_depth |   min_child_samples |   min_child_weight |   min_gain_to_split |   num_leaves |   reg_alpha |   reg_lambda |      loss |
|----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------|
| 69 |           0.962185 |             18 |           0.132123 |       0.016134  |       230 |           0 |                  93 |          0.0127495 |           0.0578221 |           42 |   0.0866981 |      39.8677 | -0.793874

Y sobre el test AUC en test: 0.796208526365374

Tras añadir finalmente todos los cambios realizado en todas las tablas:
Optimización ligera:
|    |   bagging_fraction |   bagging_freq |   feature_fraction |   learning_rate |   max_bin |   max_depth |   min_child_samples |   min_child_weight |   min_gain_to_split |   num_leaves |   reg_alpha |   reg_lambda |      loss |
|----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------|
| 24 |           0.99904  |             13 |           0.248436 |       0.0297273 |       300 |           0 |                  95 |          0.0135347 |           0.0569547 |           44 |   0.186123  |     10.5439  | -0.791107

Optimización Completa:
|    |   bagging_fraction |   bagging_freq |   feature_fraction |   learning_rate |   max_bin |   max_depth |   min_child_samples |   min_child_weight |   min_gain_to_split |   num_leaves |   reg_alpha |   reg_lambda |      loss |
|----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------|
| 69 |           0.891186 |             16 |           0.106319 |       0.0173115 |       300 |           0 |                 101 |          0.0137934 |           0.0269529 |           41 |   0.0906638 |      31.1379 | -0.794675

Y sobre el test:
AUC en test: 0.798

Si para este mismo csv V7 realizamos una permutation importance con threshold > 0 obtenemos:

| 50 |           0.940211 |             18 |           0.114142 |       0.017795  |       280 |           0 |                  97 |          0.0131429 |           0.0344192 |           35 |   0.101093  |      23.3484 | -0.794194 |

Y si probamos sobre el test:

AUC en test: 0.7955353053650761


Realizando pruebas con un rango de espacio más amplio sobre todo de cara a ver como varía la profundidad obtenemos:
| 97 |           0.985295 |             12 |           0.172079 |       0.0224613 |       275 |           3 |                 240 |          0.011828  |           0.0916892 |           2300 |          131 |   0.0738748 |      77.5059 | -0.793815 |

AUC en test: 0.795
Realizando pruebas con un espacio más acotado hemos obtenido:
-+-------------+--------------+-----------+
|    |   bagging_fraction |   bagging_freq |   feature_fraction |   learning_rate |   max_bin |   max_depth |   min_child_samples |   min_child_weight |   min_gain_to_split |   n_estimators |   num_leaves |   reg_alpha |   reg_lambda |      loss |
|----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+----------------+--------------+-------------+--------------+-----------|
| 38 |           0.960846 |             15 |           0.131173 |       0.0175334 |       295 |           4 |                 220 |          0.010008  |           0.0246018 |           2200 |          247 |   0.103971  |      46.9502 | -0.794535
Pero tras realizar pruebas sobre el test:
AUC en test: 0.7957303444675893 vemos que la mejora no es tan grande como cuando no teniamos límite de profundidad, indicando que este número excesivo de hojas posiblemente esté casuando over fitting


Mejor resultado obtenido en el csv reducido tras realizar varias pruebas. No se acerca al del csv completo. Toca ver como rinde
realizando una submisión en Kaggle. En caso negativo probar CNN para selección de variables o pasar directamente a stacking para acabar proyecto.
 AUC en test: 0.797860519764853 |
 | | | | | | |